# Lab 2: Cryptography
In this lab we will work on implementing and analysing cryptographic primitives in software.

In [ ]:
import numpy as np
np.seterr(over='ignore')
import chipwhisperer as cw
import utils
import tqdm.notebook as tqdm
import matplotlib.pyplot as plt
from reference import present

# Exercise 1: Implementing and attacking RC4
## 1a: implement RC4
Look at the wikipedia description of RC4 [here](https://en.wikipedia.org/wiki/RC4) and implement it in Python below. The test() function will validate your implementation for functional correctness.



In [ ]:
class RC4:
    def __init__(self, key: np.ndarray):
        pass
        
    def generate(self):
        # generates 1 byte of data
        pass

In [ ]:
def test(RC4, v = True):
    inputs = [[75, 101, 121], [87, 105, 107, 105], [83, 101, 99, 114, 101, 116]]
    ref_results = ["EB9F7781B734CA72A7194A2867B642950D5D4C26", 
                   "6044DB6D41B7E8E7A4D6F9FBD4428354580CB8F1", 
                   "04D46B053CA87B594172302AEC9BB9923211D435"]
    if v:
        print(f"{"Your implementation's keystream":^40} =?= {"Reference keystream":^40}")
    correct = np.zeros(len(inputs), dtype=np.bool)
    for i in range(len(inputs)):
        _input = np.array(inputs[i], dtype=np.uint8)
        rc4 = RC4(_input)
        keystream = [rc4.generate() for _ in range(20)]
        result = ''.join([f"{t:02X}" for t in keystream])
        correct[i] = (result == ref_results[i])
        if v:
            print(result, "===" if correct[i] else "=/=", ref_results[i])
    
    return np.all(correct)

In [ ]:
test(RC4, True)

## 1b: prove a bias in the second byte of RC4
Perform an analysis on your RC4 implementation to show that there is a bias in the second byte of the cipher.

# Exercise 2: Implementing PRESENT
In this second part of the lab we will implement the lightweight block cipher called [PRESENT](https://link.springer.com/chapter/10.1007/978-3-540-74735-2_31) on the Chipwhisperer NANO.

## Provided Python implementation of PRESENT
As a reference we provide you with a Python implementation of PRESENT, against which you can compare each function on the CW Nano as you implement them. As well as some test vectors (provided as part of the PRESENT publication).

In [ ]:
test_vecs = [bytearray([0x0] * 8 + [0x0] * 10),
              bytearray([0x0] * 8 + [0xff] * 10),
              bytearray([0xff] * 8 + [0x0] * 10),
              bytearray([0xff] * 8 + [0xff] * 10)]

test_cts = [0x5579C1387B228445, 0xE72C46C0F5945049, 0xA112FFC72F68417B, 0x3333DCD3213210D2]


In [ ]:
for i in range(len(test_vecs)):
    pt = int.from_bytes(test_vecs[i][:8])
    key = int.from_bytes(test_vecs[i][8:])
    ct = present.encrypt(pt, key)
    print(f"{i:2d} -- Encryption Correct: {ct == test_cts[i]} -- ciphertext = {hex(ct)} -- Decryption Correct: {present.decrypt(ct, key) == pt}")
    

## 2a: Implementing PRESENT on CW

In [ ]:
SCOPETYPE = 'CWNANO' # or CWNANO
PLATFORM = 'CWNANO'  # or CWNANO
SS_VER="SS_VER_2_1"

In [ ]:
scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial2)
prog = cw.programmers.STM32FProgrammer

In [ ]:
scope.default_setup()

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../hw/secure-sensor-v3/
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

In [ ]:
cw.program_target(scope, prog, f"../hw/secure-sensor-v3/secure-sensor-{PLATFORM}.hex")

## Testing your implementation
This section provides tests for all individual functions of the PRESENT implementation on the CW Nano, which you can use as you develop.

In [ ]:
# Tests left rotate 61
for i in range(80):
    og_val = 1 << (79 - i)
    target.send_cmd(0xaa, 0x01, bytearray(og_val.to_bytes(10)))
    pl = target.simpleserial_read(cmd='r')
    new_val = int.from_bytes(pl)
    assert(new_val == present.left_rotate(og_val, 61))

In [ ]:
# Tests key schedule
ref_key_schedule = present.generate_round_keys(0)
target.send_cmd(0xaa, 0x02, bytearray([0] * 10))
pl = target.simpleserial_read(cmd='r')

for i in range(23): # not all rounds because there is a limit on packet size
    assert(int.from_bytes(pl[i*8: (i+1)*8]) == ref_key_schedule[i])

In [ ]:
# Tests sbox layer
for i in range(16):
    og_str = np.binary_repr(i, 4) * 16
    og_val = int(og_str, 2)
    target.send_cmd(0xaa, 0x03, bytearray(og_val.to_bytes(8)))
    pl = target.simpleserial_read(cmd='s')
    new_str = np.binary_repr(int.from_bytes(pl), 64)
    for j in range(16):
        assert(new_str[4*j: 4*(j+1)] == np.binary_repr(present.sbox[i], 4))

In [ ]:
# Tests permutation layer
for i in range(62):
    og_val = 0b111 << (i)
    target.send_cmd(0xaa, 0x04, bytearray(og_val.to_bytes(8)))
    pl = target.simpleserial_read(cmd='p')
    new_val = int.from_bytes(pl)
    assert(new_val == present.permutation_layer(og_val))

In [ ]:
# Tests whole PRESENT
for i in range(len(test_vecs)):
    target.send_cmd(0xaa, 0x05, test_vecs[i])
    pl = target.simpleserial_read(cmd='p')
    print(f"{i:2d} -- Correct: {pl == test_cts[i].to_bytes(8)} -- ciphertext = {pl}")

If all tests above pass then your whole implementation should be correct! The implementation of the sensor has already been updated to use PRESENT to encrypt the sensor data as opposed to the previously existing encryption scheme. Below we perform the final test: making sure that what we decrypt using the Python implementation of PRESENT matches what was the original plaintext!

In [ ]:
# Check that applied encryption & decryption works
N = 5
enc_data = np.zeros((N, 16), dtype=np.uint8)
dec_data = np.zeros_like(enc_data)
utils.reset_target(scope)
for i in range(N):
    target.send_cmd(0x01, 0x01, bytearray([]))
    enc_data[i] = target.simpleserial_read(cmd='s')

utils.reset_target(scope)
for i in range(N):
    target.send_cmd(0x01, 0x02, bytearray([]))
    dec_data[i] = target.simpleserial_read(cmd='s')

In [ ]:
def decrypt_sensor_data(sensor_data: np.ndarray, key):
    if type(key) != int:
        key = int.from_bytes(key.tobytes())
    byte_left = int.from_bytes(sensor_data[:8].tobytes())
    byte_right = int.from_bytes(sensor_data[8:].tobytes())
    decrypted_left = present.decrypt(byte_left, key)
    decrypted_right = present.decrypt(byte_right, key)
    return np.concat((np.frombuffer((decrypted_left).to_bytes(8), dtype=np.uint8), np.frombuffer((decrypted_right).to_bytes(8), dtype=np.uint8)))

In [ ]:
# This retrieves the key from the sensor
target.send_cmd(0xdc, 0x01, bytearray([]))
key = np.array(target.simpleserial_read(cmd='k'), dtype=np.uint8)
print("key:", key)

In [ ]:
for i in range(N):
    print(i, "Correct decryption:", np.all(decrypt_sensor_data(enc_data[i], key) == dec_data[i]))
    utils.parse_sensor_data(decrypt_sensor_data(enc_data[i], key), True)
    print()

## 2b: Mode of Operation

Below shows the updated code for the encryption of sensor data:

```C
void encrypt(uint8_t *data, uint8_t data_len) {
    // encrypt first 8 bytes
    PRESENT(data, key);
    // encrypt second 8 bytes
    PRESENT(&data[8], key);
}
```

It encrypts the first 8 bytes of the sensor data, and then encrypts the second 8 bytes of the sensor data. 
- What is this mode of operation called?
- Is this a good mode of operation to use in terms of security? Why or why not?


## 2c: What is missing

In 2b you made a conclusion about the security of the mode of operation. 
- Suppose that you wanted to switch to another mode of operation (pick one which was discussed in class), what additional information would there need to be transferred to make decryption possible?
- Once the implementation is updated to a different mode of operation, are you satisfied with the security of the program/communication?

# Disconnect

In [ ]:
target.dis()
scope.dis()